
1. 환자 데이터를 처리하여 resDate, remaining_days, latitude, longitude를 계산합니다.
2. 의사의 위치와 환자 정보를 바탕으로 5km 반경 내 환자를 필터링합니다.
3. 긴급 호출 환자 및 오늘 방문 환자를 선택합니다.
4. 최종 경로를 계산하고, 각 경로 구간의 거리를 반환합니다.

In [1]:
import pandas as pd
from datetime import timedelta, date, datetime
from geopy.geocoders import Nominatim
from math import radians, sin, cos, sqrt, atan2
import time
import random

def process_and_plan_route(doctor_location, df, emergency_calls):
    """
    Processes patient data and plans the doctor's route for the day.

    Args:
        doctor_location (tuple): (latitude, longitude) of the doctor.
        df (pd.DataFrame): DataFrame containing patient data.
        emergency_calls (list): List of patient IDs requiring emergency attention.

    Returns:
        dict: Route plan with patient details and distances between visits.
    """
    # Initialize geolocator
    geolocator = Nominatim(user_agent="South Korea")

    # Function to calculate next visit date
    def calculate_next_visit(row):
        prev_visit_date = pd.to_datetime(row['visitDate'])
        visit_freq = row['period']

        if visit_freq == 'Every 6 months':
            next_visit_date = prev_visit_date + timedelta(days=180)
        elif visit_freq == 'Every 1 year':
            next_visit_date = prev_visit_date + timedelta(days=365)
        elif visit_freq == 'Every 3 months':
            next_visit_date = prev_visit_date + timedelta(days=90)
        elif visit_freq == 'Every 2 months':
            next_visit_date = prev_visit_date + timedelta(days=60)
        else:
            next_visit_date = prev_visit_date

        return next_visit_date.strftime('%Y-%m-%d')

    # Add 'resDate' and 'remaining_days' columns
    df['resDate'] = df.apply(calculate_next_visit, axis=1)
    df['resDate'] = pd.to_datetime(df['resDate'])
    today = pd.to_datetime(date.today())
    df['remaining_days'] = (df['resDate'] - today).dt.days

    # Add latitude and longitude based on address
    df['latitude'] = None
    df['longitude'] = None
    for index, row in df.iterrows():
        address = row['address']
        try:
            location = geolocator.geocode(address)
            if location:
                df.at[index, 'latitude'] = location.latitude
                df.at[index, 'longitude'] = location.longitude
            else:
                df.at[index, 'latitude'] = None
                df.at[index, 'longitude'] = None
        except Exception as e:
            print(f"Error geocoding {address}: {e}")
            df.at[index, 'latitude'] = None
            df.at[index, 'longitude'] = None
        time.sleep(1)

    # Function to calculate distance between two points
    def calculate_distance(lat1, lon1, lat2, lon2):
        R = 6371  # Earth's radius in km
        lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
        dlat = lat2 - lat1
        dlon = lon2 - lon1
        a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))
        return R * c

    # Filter patients within 5km radius
    nearby_patients = []
    for _, row in df.iterrows():
        if row['latitude'] and row['longitude']:
            distance = calculate_distance(
                doctor_location[0], doctor_location[1],
                row['latitude'], row['longitude']
            )
            if distance <= 5:
                nearby_patients.append({
                    'patientid': row['patientid'],
                    'remaining_days': row['remaining_days'],
                    'location': (row['latitude'], row['longitude'])
                })

    # Separate emergency patients
    emergency_patients = [p for p in nearby_patients if p['patientid'] in emergency_calls]

    # Sort by remaining_days
    nearby_patients.sort(key=lambda x: x['remaining_days'])

    # Randomize the total patients to visit
    random.seed(datetime.now().strftime("%Y%m%d"))  # Use current date as seed
    total_patients_needed = random.randint(4, 7)

    # Plan the route
    if len(emergency_patients) >= total_patients_needed:
        route_plan = emergency_patients[:total_patients_needed]
    else:
        remaining_slots = total_patients_needed - len(emergency_patients)
        regular_patients = [p for p in nearby_patients if p not in emergency_patients]
        route_plan = emergency_patients + regular_patients[:remaining_slots]

    # Calculate route distances
    route_distances = []
    if route_plan:
        # Distance from doctor to first patient
        route_distances.append(calculate_distance(
            doctor_location[0], doctor_location[1],
            route_plan[0]['location'][0], route_plan[0]['location'][1]
        ))
        # Distance between subsequent patients
        for i in range(len(route_plan) - 1):
            route_distances.append(calculate_distance(
                route_plan[i]['location'][0], route_plan[i]['location'][1],
                route_plan[i + 1]['location'][0], route_plan[i + 1]['location'][1]
            ))

    return {
        'route_plan': route_plan,
        'route_distances': route_distances
    }


In [2]:
import pandas as pd
import random
from geopy.geocoders import Nominatim

# Geopy 초기화
geolocator = Nominatim(user_agent="geoapi")

# 성남시 수정구 태평동 근처 좌표 설정
base_latitude = 37.4409  # 성남시 수정구 태평동 근처 위도
base_longitude = 127.1376  # 성남시 수정구 태평동 근처 경도

# 랜덤 좌표 생성 함수
def generate_random_coordinates(base_lat, base_lon, radius=0.01):
    lat = base_lat + random.uniform(-radius, radius)  # 약 1km 반경
    lon = base_lon + random.uniform(-radius, radius)
    return lat, lon

# 좌표를 주소로 변환하는 함수
def get_address_from_coordinates(lat, lon):
    try:
        location = geolocator.reverse((lat, lon), exactly_one=True, language="ko")
        return location.address if location else "Unknown address"
    except Exception as e:
        return "Geocoding failed"

# 데이터 생성
data = []
for patient_id in range(1, 16):
    lat, lon = generate_random_coordinates(base_latitude, base_longitude)
    address = get_address_from_coordinates(lat, lon)
    data.append({
        "patientid": patient_id,
        "age": random.randint(20, 80),
        "gender": random.choice(["Male", "Female"]),
        "bloodType": random.choice(["A", "B", "O", "AB"]),
        "diseases": random.choice(["Diabetes", "Asthma", "Hypertension", "Cancer", "Obesity", "Arthritis"]),
        "period": random.choice(["Every 1 year", "Every 6 months", "Every 3 months", "Every 2 months"]),
        "visitDate": pd.Timestamp('2024-01-01') + pd.to_timedelta(random.randint(0, 365), unit='d'),
        "address": address
    })

# 데이터프레임 생성
sam = pd.DataFrame(data)
sam

,patientid,age,gender,bloodType,diseases,period,visitDate,address
0,1,65,Male,AB,Diabetes,Every 6 months,2024-09-12,"가천대학교 글로벌캠퍼스, 복정로32번길, 복정동, 수정구, 성남시, 경기도, 131..."
1,2,77,Female,B,Diabetes,Every 2 months,2024-06-21,"1176, 성남대로, 수진동, 수정구, 성남시, 경기도, 13326, 대한민국"
2,3,66,Male,B,Obesity,Every 6 months,2024-01-29,"남문로89번길, 태평동, 수정구, 성남시, 경기도, 13285, 대한민국"
3,4,77,Female,AB,Arthritis,Every 6 months,2024-05-04,"모란고가교, 모란로, 수진동, 수정구, 성남시, 경기도, 13313, 대한민국"
4,5,50,Male,B,Obesity,Every 6 months,2024-05-03,"신지교회.하늘채아파트, 시민로, 하대원동, 밭, 중원구, 성남시, 경기도, 1335..."
5,6,39,Male,AB,Diabetes,Every 3 months,2024-12-05,"산성대로, 중앙동, 중원구, 성남시, 경기도, 13354, 대한민국"
6,7,42,Female,AB,Hypertension,Every 1 year,2024-11-15,"수정로136번길, 태평동, 수정구, 성남시, 경기도, 13327, 대한민국"
7,8,36,Male,AB,Asthma,Every 1 year,2024-11-05,"17-1, 시민로175번길, 신흥동, 수정구, 성남시, 경기도, 13338, 대한민국"
8,9,67,Male,AB,Diabetes,Every 2 months,2024-08-17,"시민로261번길, 태평동, 수정구, 성남시, 경기도, 13285, 대한민국"
9,10,44,Male,B,Cancer,Every 1 year,2024-10-05,"수정북로, 신흥동, 수정구, 성남시, 경기도, 13291, 대한민국"


In [4]:
result = process_and_plan_route((37.4409, 127.1376), sam, [1, 2, 3])
result

{'route_plan': [{'patientid': 1,
   'remaining_days': 109,
   'location': (37.452797950000004, 127.13377749227436)},
  {'patientid': 2,
   'remaining_days': -94,
   'location': (37.4348713, 127.1291267)},
  {'patientid': 3,
   'remaining_days': -118,
   'location': (37.448232653672214, 127.13598000790354)},
  {'patientid': 14,
   'remaining_days': -102,
   'location': (37.45069995, 127.12853341575823)},
  {'patientid': 13,
   'remaining_days': -94,
   'location': (37.43732972920486, 127.1363952043217)}],
 'route_distances': [1.3653494680964662,
  2.0351991978990505,
  1.6041926690101755,
  0.7123129169556695,
  1.6407318841115193]}

# 모델 배포

In [13]:
import ssl
from geopy.geocoders import Nominatim

# SSL 인증서를 무시하는 설정
context = ssl.create_default_context()
context.check_hostname = False
context.verify_mode = ssl.CERT_NONE

# Nominatim 객체에 SSL context 추가
geolocator = Nominatim(user_agent="South Korea", ssl_context=context)

# 예시 사용
location = geolocator.geocode("서울특별시")
print(location.latitude, location.longitude)


37.5666791 126.9782914


In [18]:
import pandas as pd
from datetime import timedelta, date, datetime
from geopy.geocoders import Nominatim
from math import radians, sin, cos, sqrt, atan2
import time
import random
import pickle

class RoutePlanner:
    def __init__(self, doctor_location, df, emergency_calls):
        self.doctor_location = doctor_location  # 의사의 위치 (위도, 경도)
        self.df = df  # 환자 데이터
        self.emergency_calls = emergency_calls  # 응급 환자 목록
        self.geolocator = Nominatim(user_agent="South Korea")  # Geolocator 초기화

    def calculate_next_visit(self, row):
        """
        환자의 다음 방문 일자를 계산하는 함수
        """
        prev_visit_date = pd.to_datetime(row['visitDate'])
        visit_freq = row['period']

        if visit_freq == 'Every 6 months':
            next_visit_date = prev_visit_date + timedelta(days=180)
        elif visit_freq == 'Every 1 year':
            next_visit_date = prev_visit_date + timedelta(days=365)
        elif visit_freq == 'Every 3 months':
            next_visit_date = prev_visit_date + timedelta(days=90)
        elif visit_freq == 'Every 2 months':
            next_visit_date = prev_visit_date + timedelta(days=60)
        else:
            next_visit_date = prev_visit_date

        return next_visit_date.strftime('%Y-%m-%d')

    def add_geolocation(self):
        """
        환자의 주소를 기반으로 위도와 경도 추가하는 함수
        """
        self.df['latitude'] = None
        self.df['longitude'] = None
        for index, row in self.df.iterrows():
            address = row['address']
            try:
                location = self.geolocator.geocode(address)
                if location:
                    self.df.at[index, 'latitude'] = location.latitude
                    self.df.at[index, 'longitude'] = location.longitude
                else:
                    self.df.at[index, 'latitude'] = None
                    self.df.at[index, 'longitude'] = None
            except Exception as e:
                print(f"Error geocoding {address}: {e}")
                self.df.at[index, 'latitude'] = None
                self.df.at[index, 'longitude'] = None
            time.sleep(1)

    def calculate_distance(self, lat1, lon1, lat2, lon2):
        """
        두 지점 간의 거리를 계산하는 함수
        """
        R = 6371  # Earth's radius in km
        lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
        dlat = lat2 - lat1
        dlon = lon2 - lon1
        a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))
        return R * c

    def process_and_plan_route(self):
        """
        환자 데이터를 처리하고 의사의 경로를 계획하는 함수
        """
        # 다음 방문일 계산
        self.df['resDate'] = self.df.apply(self.calculate_next_visit, axis=1)
        self.df['resDate'] = pd.to_datetime(self.df['resDate'])
        today = pd.to_datetime(date.today())
        self.df['remaining_days'] = (self.df['resDate'] - today).dt.days

        # 환자 주소로부터 위도, 경도 추가
        self.add_geolocation()

        # 5km 반경 내 환자들 필터링
        nearby_patients = []
        for _, row in self.df.iterrows():
            if row['latitude'] and row['longitude']:
                distance = self.calculate_distance(
                    self.doctor_location[0], self.doctor_location[1],
                    row['latitude'], row['longitude']
                )
                if distance <= 5:
                    nearby_patients.append({
                        'patientid': row['patientid'],
                        'remaining_days': row['remaining_days'],
                        'location': (row['latitude'], row['longitude'])
                    })

        # 응급 환자 별도 처리
        emergency_patients = [p for p in nearby_patients if p['patientid'] in self.emergency_calls]

        # 남은 환자들 정렬 (남은 방문일 기준)
        nearby_patients.sort(key=lambda x: x['remaining_days'])

        # 방문할 환자 수 랜덤화
        random.seed(datetime.now().strftime("%Y%m%d"))
        total_patients_needed = random.randint(4, 7)

        # 경로 계획
        if len(emergency_patients) >= total_patients_needed:
            route_plan = emergency_patients[:total_patients_needed]
        else:
            remaining_slots = total_patients_needed - len(emergency_patients)
            regular_patients = [p for p in nearby_patients if p not in emergency_patients]
            route_plan = emergency_patients + regular_patients[:remaining_slots]

        # 경로 거리 계산
        route_distances = []
        if route_plan:
            # 의사에서 첫 환자까지 거리
            route_distances.append(self.calculate_distance(
                self.doctor_location[0], self.doctor_location[1],
                route_plan[0]['location'][0], route_plan[0]['location'][1]
            ))
            # 이후 환자 간 거리
            for i in range(len(route_plan) - 1):
                route_distances.append(self.calculate_distance(
                    route_plan[i]['location'][0], route_plan[i]['location'][1],
                    route_plan[i + 1]['location'][0], route_plan[i + 1]['location'][1]
                ))

        return {
            'route_plan': route_plan,
            'route_distances': route_distances
        }

# 객체 저장
def save_route_planner(route_planner, filename="route_planner.pkl"):
    with open(filename, 'wb') as file:
        pickle.dump(route_planner, file)

# 객체 불러오기
def load_route_planner(filename="route_planner.pkl"):
    with open(filename, 'rb') as file:
        return pickle.load(file)

# 예시 사용
# doctor_location = (37.5665, 126.9780)  # 서울의 예시 위치
# df = pd.DataFrame(...)  # 환자 데이터프레임
# emergency_calls = [...]  # 응급 환자 목록

# route_planner = RoutePlanner(doctor_location, df, emergency_calls)
# route_plan = route_planner.process_and_plan_route()
# save_route_planner(route_planner)

# 불러오기 예시
# loaded_planner = load_route_planner()
# print(loaded_planner.process_and_plan_route())


In [30]:
from flask import Flask, request, jsonify
import pickle

app = Flask(__name__)

# 모델 로드
with open('/content/route_planner_model.pkl', 'rb') as f:
    model = pickle.load(f)

@app.route('/predict', methods=['POST'])
def predict():
    data = request.get_json()
    df = pd.DataFrame(data['patients'])  # 환자 데이터
    emergency_calls = data['emergency_calls']  # 응급 환자 리스트

    # 모델 예측
    result = model.plan_route(df, emergency_calls)

    return jsonify(result)

if __name__ == '__main__':
    app.run(debug=True)


AttributeError: 'RequestsHTTPWithSSLContextAdapter' object has no attribute '_RequestsHTTPWithSSLContextAdapter__ssl_context'

In [26]:
from geopy.geocoders import Nominatim
from requests.adapters import HTTPAdapter
import ssl
import requests

# Nominatim geocoder 설정
geolocator = Nominatim(user_agent="South Korea")

# SSL 검증 비활성화
session = requests.Session()
adapter = HTTPAdapter(max_retries=3)
session.mount('https://', adapter)

# SSL 컨텍스트를 안전하지 않게 설정
session.verify = False
geolocator.session = session



In [28]:
pip install --upgrade geopy


# 최종 최적경로 함수 적용

In [37]:
import pandas as pd
import random
from math import radians, sin, cos, sqrt, atan2
from datetime import datetime, timedelta, date
from geopy.geocoders import Nominatim
import time

def process_and_plan_route(df, doctor_coords, additional_calls):
    """
    Processes patient data and plans the optimal route for the doctor.

    Args:
        df (pd.DataFrame): DataFrame with patient data.
        doctor_coords (tuple): Tuple of latitude and longitude for the doctor (e.g., (37.4409, 127.1376)).
        additional_calls (list): List of patient IDs for additional visits.

    Returns:
        list: Optimal route as a list of locations (latitude, longitude).
    """
    # Initialize geolocator
    geolocator = Nominatim(user_agent="South Korea")

    # Helper function: Calculate next visit date
    def calculate_next_visit(row):
        """Calculates the next visit date based on the visit frequency."""
        prev_visit_date = pd.to_datetime(row['visitDate'])
        visit_freq = row['period']
        if visit_freq == 'Every 6 months':
            next_visit_date = prev_visit_date + timedelta(days=180)
        elif visit_freq == 'Every 1 year':
            next_visit_date = prev_visit_date + timedelta(days=365)
        elif visit_freq == 'Every 3 months':
            next_visit_date = prev_visit_date + timedelta(days=90)
        elif visit_freq == 'Every 2 months':
            next_visit_date = prev_visit_date + timedelta(days=60)
        else:
            next_visit_date = prev_visit_date  # Default case
        return next_visit_date.strftime('%Y-%m-%d')

    # Helper function: Calculate distance between two points
    def calculate_distance(point1, point2):
        R = 6371  # Earth's radius in kilometers
        lat1, lon1 = point1
        lat2, lon2 = point2
        lat1, lon1 = radians(lat1), radians(lon1)
        lat2, lon2 = radians(lat2), radians(lon2)
        dlat = lat2 - lat1
        dlon = lon2 - lon1
        a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))
        return R * c

    # Helper function: Find the shortest path (simplified nearest neighbor approach)
    def find_shortest_path(start, locations):
        if not locations:
            return [], 0
        path = [start]
        total_distance = 0
        remaining = locations[:]
        current = start
        while remaining:
            nearest = min(remaining, key=lambda loc: calculate_distance(current, loc))
            total_distance += calculate_distance(current, nearest)
            path.append(nearest)
            remaining.remove(nearest)
            current = nearest
        return path, total_distance

    # Step 1: Process patient data
    df['resDate'] = df.apply(calculate_next_visit, axis=1)
    df['resDate'] = pd.to_datetime(df['resDate'])
    today = pd.to_datetime(date.today())
    df['remaining_days'] = (df['resDate'] - today).dt.days

    df['latitude'] = None
    df['longitude'] = None
    for index, row in df.iterrows():
        address = row['address']
        try:
            location = geolocator.geocode(address)
            if location:
                df.at[index, 'latitude'] = location.latitude
                df.at[index, 'longitude'] = location.longitude
            else:
                df.at[index, 'latitude'] = None
                df.at[index, 'longitude'] = None
        except Exception as e:
            print(f"Error geocoding {address}: {e}")
            df.at[index, 'latitude'] = None
            df.at[index, 'longitude'] = None
        time.sleep(1)

    # Step 2: Classify patients
    patients = df.to_dict('records')
    priority_patients, additional_patients, normal_patients = [], [], []
    for patient in patients:
        if pd.isna(patient['latitude']) or pd.isna(patient['longitude']):
            continue
        patient_coords = (patient['latitude'], patient['longitude'])
        distance = calculate_distance(doctor_coords, patient_coords)
        if distance <= 5:  # Only consider patients within 5 km
            if patient['remaining_days'] <= 0:
                priority_patients.append({'id': patient['patientid'], 'location': patient_coords})
            elif patient['patientid'] in additional_calls:
                additional_patients.append({'id': patient['patientid'], 'location': patient_coords})
            else:
                normal_patients.append({'id': patient['patientid'], 'location': patient_coords})

    # Step 3: Combine patient lists
    combined_list = priority_patients + additional_patients + normal_patients
    random.seed(date.today().toordinal())
    patients_count = random.randint(4, 7)
    route_plan = combined_list[:min(patients_count, len(combined_list))]

    # Step 4: Calculate optimal route
    priority_locations = [p['location'] for p in route_plan if p in priority_patients]
    best_route, _ = find_shortest_path(doctor_coords, priority_locations)

    if any(p in additional_patients for p in route_plan):
        additional_locations = [p['location'] for p in route_plan if p in additional_patients]
        additional_path, _ = find_shortest_path(best_route[-1], additional_locations)
        best_route.extend(additional_path)

    if any(p in normal_patients for p in route_plan):
        normal_locations = [p['location'] for p in route_plan if p in normal_patients]
        normal_path, _ = find_shortest_path(best_route[-1], normal_locations)
        best_route.extend(normal_path)

    return best_route


In [41]:
# 의사 위치 (위도, 경도)
doctor_coords = (37.4409, 127.1376)

# 추가 방문 대상자 ID
additional_calls = [2]

# 함수 실행
route = process_and_plan_route(sam, doctor_coords, additional_calls)

# 경로 출력
print(route)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(37.4409, 127.1376), (37.43732972920486, 127.1363952043217), (37.4348713, 127.1291267), (37.4333582, 127.12790880209067), (37.436983, 127.1464419), (37.448232653672214, 127.13598000790354), (37.4503054125531, 127.13706399022698), (37.45069995, 127.12853341575823)]


In [45]:
#벡엔드 전달 데이터 distance
distance = []
distance.append(int(calculate_distance(doctor_coords, route[0])*1000))
for i in range(1, len(route)):
		distance.append(int(calculate_distance(route[i-1], route[i])*1000))

TypeError: calculate_distance() missing 2 required positional arguments: 'lat2' and 'lon2'

In [46]:
#벡엔드 전달 데이터 distance
distance = []
# The original calculate_distance function is designed to take two points as tuples
# doctor_coords is already a tuple, so we need to ensure route[0] is a tuple as well.
distance.append(int(calculate_distance(doctor_coords, route[0])*1000))
for i in range(1, len(route)):
    # Similar to the above, both route[i-1] and route[i] should be tuples.
    distance.append(int(calculate_distance(route[i-1], route[i])*1000))

TypeError: calculate_distance() missing 2 required positional arguments: 'lat2' and 'lon2'

#FastAPI 코드 작성

In [50]:
from fastapi import FastAPI, File, UploadFile
from fastapi.responses import JSONResponse
from fastapi.middleware.cors import CORSMiddleware
import pandas as pd
import json
from typing import List
from datetime import date
from math import radians, sin, cos, sqrt, atan2
from geopy.geocoders import Nominatim
import time

# Colab에서 작성된 함수 (간소화된 형태)
def process_and_plan_route(df, doctor_coords, additional_calls):
    """
    Processes patient data and plans the optimal route for the doctor.
    Args:
        df (pd.DataFrame): DataFrame containing patient data.
        doctor_coords (tuple): Doctor's latitude and longitude.
        additional_calls (list): List of additional patient IDs to visit.
    Returns:
        list: Processed patient data with calculated distances.
    """
    # Initialize geolocator
    geolocator = Nominatim(user_agent="South Korea")

    def calculate_distance(point1, point2):
        """Calculate distance between two latitude/longitude points."""
        R = 6371  # Earth's radius in kilometers
        lat1, lon1 = point1
        lat2, lon2 = point2
        lat1, lon1 = radians(lat1), radians(lon1)
        lat2, lon2 = radians(lat2), radians(lon2)
        dlat = lat2 - lat1
        dlon = lon2 - lon1
        a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))
        return R * c

    # Add latitude and longitude from address
    df['latitude'] = None
    df['longitude'] = None
    for index, row in df.iterrows():
        try:
            location = geolocator.geocode(row['address'])
            if location:
                df.at[index, 'latitude'] = location.latitude
                df.at[index, 'longitude'] = location.longitude
        except Exception as e:
            print(f"Error geocoding {row['address']}: {e}")
            df.at[index, 'latitude'] = None
            df.at[index, 'longitude'] = None
        time.sleep(1)

    # Calculate distances and classify patients
    df['distance_from_doctor'] = df.apply(
        lambda row: calculate_distance(doctor_coords, (row['latitude'], row['longitude']))
        if row['latitude'] and row['longitude']
        else None, axis=1
    )

    df['priority'] = df['patientid'].apply(
        lambda x: 'priority' if x in additional_calls else 'normal'
    )

    return df.to_dict(orient='records')

# FastAPI 인스턴스 생성
app = FastAPI()

# CORS 설정
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

@app.post("/process-patient-data/")
async def process_patient_data(
    file: UploadFile,
    doctor_lat: float,
    doctor_lon: float,
    additional_calls: List[int]
):
    """
    Processes uploaded patient data JSON and returns the result.
    Args:
        file (UploadFile): Uploaded JSON file containing patient data.
        doctor_lat (float): Doctor's latitude.
        doctor_lon (float): Doctor's longitude.
        additional_calls (list): List of additional patient IDs for visits.
    Returns:
        JSONResponse: Processed patient data.
    """
    try:
        # Load JSON file into a DataFrame
        content = await file.read()
        data = json.loads(content)
        df = pd.DataFrame(data)
    except Exception as e:
        return JSONResponse(content={"error": f"Failed to read JSON file: {str(e)}"}, status_code=400)

    # Process data using Colab-defined function
    try:
        doctor_coords = (doctor_lat, doctor_lon)
        processed_data = process_and_plan_route(df, doctor_coords, additional_calls)
    except Exception as e:
        return JSONResponse(content={"error": f"Data processing failed: {str(e)}"}, status_code=500)

    # Return processed data as JSON
    return JSONResponse(content=processed_data)


In [49]:
pip install python-multipart


In [52]:
pip install pyngrok

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [53]:
from pyngrok import ngrok

# FastAPI 실행 파일 이름: example.py
!uvicorn example:app --host 0.0.0.0 --port 8000 &

# ngrok으로 외부 접속 URL 생성
public_url = ngrok.connect(8000)
print(f"Public URL: {public_url}")

ERROR:    Error loading ASGI app. Could not import module "example".


ERROR:pyngrok.process.ngrok:t=2024-11-22T23:42:01+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2024-11-22T23:42:01+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2024-11-22T23:42:01+0000 lvl=eror msg="terminating with error" obj=app err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your aut

PyngrokNgrokError: The ngrok process errored on start: authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.